# Pytorch Based Convolutional Autoencoders For Anomaly Detection
useful links:
- [Base autoencoder model](https://www.geeksforgeeks.org/denoising-autoencoders-in-machine-learning/)
- [Batch Normalization Explained](https://towardsdatascience.com/batch-norm-explained-visually-how-it-works-and-why-neural-networks-need-it-b18919692739)
- [Build a Neural Network in Pytorch](https://towardsdatascience.com/three-ways-to-build-a-neural-network-in-pytorch-8cea49f9a61a)
- [Anomaly Detection With IQR](https://builtin.com/articles/1-5-iqr-rule)

## Initial Setup

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch

from torch import nn, optim
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets
from torchvision.transforms import v2
from torchinfo import summary
from tqdm.auto import tqdm

/home/nata-brain/miniconda3/envs/mpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Check cuda availability

In [10]:
torch.cuda.is_available()

True

# Set Parameters

In [21]:
# Env Parameters
DATA_DIR='../datasets/cable_dataset_tester/images/dae_dataset'
MODEL_OUT='../models'
GRAPH_OUT='results'
NET_NAME='cable_novelty_v2.pt'

# Hyperparameters
DEVICE = 'cuda'
print(DEVICE)
IMG_SHAPE = (300, 600, 1)

BATCH_SIZE = 32 # originaly 16
LATENT_DIM = 2
LR = 3e-4
EPOCHS = 5000

os.makedirs(MODEL_OUT, exist_ok=True)
os.makedirs(GRAPH_OUT, exist_ok=True)

cuda


## Neural Network Definition

In [22]:
class DAE(nn.Module):
    def __init__(self, img_shape: tuple, latent_dim: int) -> None:
        super(DAE, self).__init__()
        self.img_shape = img_shape
        
        # Deep Neural Network
        self.linear1 = nn.Linear(np.prod(img_shape), 128) # Applies an affine linear transformation to the incoming data: y = x.A^T + b.
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, 512)
        self.linear4 = nn.Linear(512, 256)
        self.linear5 = nn.Linear(256, 128)
        self.linear6 = nn.Linear(128, np.prod(img_shape))
        self.latent1 = nn.Linear(512, latent_dim)
        self.latent2 = nn.Linear(latent_dim, 512)

        self.relu = nn.LeakyReLU(negative_slope=.1, inplace=True)
        self.tanh = nn.Tanh()
    
    def encode(self, x):
        """
        Encoder Creation
        
        Args: 
            x - pixel value
            
        Return:
            Vector encoded
        """
        
        h1 = self.relu(self.linear1(x))
        h2 = self.relu(self.linear2(h1))
        
        return self.relu(self.linear3(h2)) # Encoded data
    
    def decoder(self, z):
        """
        Decoder Creation
        
        Args: 
            z - pixel value
            
        Return:
            Vector decoded 
        """
        
        h4 = self.relu(self.linear4(z))
        h5 = self.relu(self.linear5(h4))
        
        return self.tanh(self.linear6(h5)) # Decoded data
        
    def forward(self, img):
        """
        Feedfoward function to get the reconstructed image
        
        Args: 
            img - image vector
            
        Return:
            Reconstructed vector 
        """
        
        og_shape = img.shape
        img_flat = img.view(img.shape[0], -1)
        z = self.encode(img_flat)
        recon = self.decoder(z)
        
        return recon.view(og_shape) # Reconstructed image vector

# Model creation 

In [23]:
model = DAE(IMG_SHAPE, LATENT_DIM).to(DEVICE)
print(DEVICE)

cuda


In [24]:
summary(model, input_size=(BATCH_SIZE, 1, 600, 300))

Layer (type:depth-idx)                   Output Shape              Param #
DAE                                      [32, 1, 600, 300]         2,562
├─Linear: 1-1                            [32, 128]                 23,040,128
├─LeakyReLU: 1-2                         [32, 128]                 --
├─Linear: 1-3                            [32, 256]                 33,024
├─LeakyReLU: 1-4                         [32, 256]                 --
├─Linear: 1-5                            [32, 512]                 131,584
├─LeakyReLU: 1-6                         [32, 512]                 --
├─Linear: 1-7                            [32, 256]                 131,328
├─LeakyReLU: 1-8                         [32, 256]                 --
├─Linear: 1-9                            [32, 128]                 32,896
├─LeakyReLU: 1-10                        [32, 128]                 --
├─Linear: 1-11                           [32, 180000]              23,220,000
├─Tanh: 1-12                             [32, 18

In [25]:
# loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [26]:
class EarlyStopping:
    def __init__(self, patience:int=5, min_delta:int=0) -> None:
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_val_loss = float('inf')
    
    def early_stop(self, val_loss):
        if val_loss < self.min_val_loss:
            self.min_val_loss = val_loss
            self.counter = 0
        elif val_loss > (self.min_val_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

## Loading The Data

In [27]:
img_transforms = v2.Compose([
    v2.Resize(IMG_SHAPE[:-1]),
    v2.Grayscale(),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)
])

train_set = datasets.ImageFolder(
    os.path.join(DATA_DIR, 'train'), 
    transform=img_transforms)

train_set, val_set = random_split(train_set, [.7,.3])

In [28]:
train_loader = DataLoader(train_set, BATCH_SIZE, shuffle=True, drop_last=True)
val_loader = DataLoader(val_set, BATCH_SIZE, shuffle=False)

## Training

In [29]:
class AutoSave:
    def __init__(self, filename: str) -> None:
        if not filename.endswith('.pt'):
            filename += '.pt'
        self.filename = filename
        self.best_loss = float('inf')
        
    def check_save(self, model, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            torch.save(model.state_dict(), 
                       os.path.join(MODEL_OUT, self.filename))

In [30]:
early_stopping = EarlyStopping(10, .0001)
auto_save = AutoSave(NET_NAME)
history = {'train_loss': [], 'val_loss': []}

# Training Loop
pbar = tqdm(range(EPOCHS), desc='Model Training Progress', position=0)
for epoch in pbar:
    model.train(True)
    train_loss = 0
    val_loss = 0
    
    for i, (data, _) in enumerate(train_loader):
        data = data.to(DEVICE)
        optimizer.zero_grad()
        
        data_noise = torch.randn(data.shape).to(DEVICE)
        data_noise = data + data_noise
        
        recon_batch = model(data_noise.to(DEVICE))
        loss = criterion(recon_batch, data)
        loss.backward()
        
        train_loss += loss.item() * len(data)
        optimizer.step()
        
        pbar.set_postfix({'Step': i+1, 'Train Loss': train_loss/(i+1)})
        
    train_loss = train_loss / len(train_loader)
    model.eval()
    
    with torch.no_grad():
        for j, (vdata, _) in enumerate(val_loader):
            vdata = vdata.to(DEVICE)
            
            vdata_noise = torch.randn(vdata.shape).to(DEVICE)
            vdata_noise = vdata + vdata_noise
            
            vouts = model(vdata_noise.to(DEVICE))
            vloss = criterion(vouts, vdata)
            
            val_loss += vloss.item() * len(vdata)
            
            pbar.set_postfix({
                'Train Loss': train_loss, 
                'Validation Loss': val_loss / (j+1)
            })
            
    val_loss = val_loss / len(val_loader)
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    pbar.write(f'Epoch {epoch+1}/{EPOCHS} -> Train loss: {train_loss}, \
        Validation loss: {val_loss}')
    
    auto_save.check_save(model, val_loss)
    
    if early_stopping.early_stop(val_loss):
        print(f'Validation loss did not improve for \
        {early_stopping.patience} epochs, interrupting training.')
        break
    
# pbar.write(f'Final Stats -> Train Loss: {train_loss}, Validation Loss: {val_loss}')

Model Training Progress:   0%|          | 1/5000 [00:24<34:31:29, 24.86s/it, Train Loss=1.19, Validation Loss=0.693]

Epoch 1/5000 -> Train loss: 1.1931875901435738,         Validation loss: 0.693364061622156


Model Training Progress:   0%|          | 1/5000 [00:48<34:31:29, 24.86s/it, Train Loss=0.632, Validation Loss=0.548]

Epoch 2/5000 -> Train loss: 0.632257802006024,         Validation loss: 0.5478468073868297


Model Training Progress:   0%|          | 2/5000 [01:15<35:11:30, 25.35s/it, Train Loss=0.508, Validation Loss=0.457]

...

Epoch 198/5000 -> Train loss: 0.11814046417599294,         Validation loss: 0.12284944681030335


Model Training Progress:   4%|▍         | 199/5000 [1:22:26<31:19:15, 23.49s/it, Step=3, Train Loss=0.13]                

Epoch 199/5000 -> Train loss: 0.11805163034752233,         Validation loss: 0.12318300560097366


Model Training Progress:   4%|▍         | 199/5000 [1:22:50<33:18:37, 24.98s/it, Train Loss=0.118, Validation Loss=0.123]

Epoch 200/5000 -> Train loss: 0.1179715160113662,         Validation loss: 0.12280582807498933
Validation loss did not improve for         10 epochs, interrupting training.


## Results





















In [ ]:
sns.lineplot(history)

In [ ]:
test_set = datasets.ImageFolder(
    os.path.join(DATA_DIR, 'test'),
    transform=img_transforms,
)

indices_bad = (torch.tensor(test_set.targets)[..., None] == 0).any(-1).nonzero(as_tuple=True)[0]
indices_good = (torch.tensor(test_set.targets)[..., None] == 1).any(-1).nonzero(as_tuple=True)[0]

test_set_good = Subset(test_set, indices_good)
test_set_bad = Subset(test_set, indices_bad)

test_loader_good = DataLoader(test_set_good, len(indices_good))
test_loader_bad = DataLoader(test_set_bad, len(indices_bad))

In [ ]:

with torch.no_grad():
    for data, _ in test_loader_bad:
        data = data.to(DEVICE)
        recon = model(data)
        break

import matplotlib.pyplot as plt
plt.figure(dpi=250)
fig, ax = plt.subplots(2, 7, figsize=(15, 4))
for i in range(7):
    ax[0, i].imshow(data[i].cpu().numpy().transpose((1, 2, 0)))
    ax[1, i].imshow(recon[i].cpu().numpy().transpose((1, 2, 0)))
    ax[0, i].axis('OFF')
    ax[1, i].axis('OFF')
plt.show()


In [ ]:
fig.savefig(os.path.join(GRAPH_OUT, NET_NAME + '_anom_sample.png'))

In [ ]:
def pearson_correlation(p1, p2):
    return np.corrcoef(p1.flatten(), p2.flatten(), rowvar=False)[0][1]

good_corr = list(map(pearson_correlation, data.cpu().numpy(), recon.cpu().numpy()))

print(f'Pearson Correlation Coefficients:\n\
    Min: {np.min(good_corr)}\n\
    Median: {np.median(good_corr)}\n\
    Max: {np.max(good_corr)}')

In [ ]:
def euclidean_dist(p1, p2):
    return np.linalg.norm(p1-p2)

dists = list(map(euclidean_dist, data.cpu().numpy(), recon.cpu().numpy()))

print(f'Euclidean Distances:\n\
    Min: {np.min(dists)}\n\
    Median: {np.median(dists)}\n\
    Max: {np.max(dists)}')

In [ ]:
def root_mean_squared_error(p1, p2):
    return np.square(np.subtract(p1, p2)).mean()

def stats(arr):
    """Calculates the IQR of array elements.

    Args:
        arr (np.array): array with numeric values

    Returns:
        float: IQR calculated from quartiles
        float: lower bound for anomaly detection
        float: higher bound for anomaly detection
    """
    q75, q25 = np.percentile(arr, [75, 25])
    iqr = q75 - q25
    return iqr, q25 - 1.5 * iqr, q75 + 1.5 * iqr

iqr, lower_bound, higher_bound = stats(list(map(pearson_correlation, data.cpu().numpy(), recon.cpu().numpy())))

print(f'IQR: {iqr}\nLower Bound: {lower_bound}\nHigher Bound: {higher_bound}')

In [ ]:
with torch.no_grad():
    for img, _ in test_loader_good:
        img = img.to(DEVICE)
        recons = model(img)
        break

plt.figure(dpi=250)
fig, ax = plt.subplots(2, 7, figsize=(15, 4))
for i in range(7):
    ax[0, i].imshow(img[i].cpu().numpy().transpose((1, 2, 0)))
    ax[1, i].imshow(recons[i].cpu().numpy().transpose((1, 2, 0)))
    ax[0, i].axis('OFF')
    ax[1, i].axis('OFF')
plt.show()

In [ ]:
fig.savefig(os.path.join(GRAPH_OUT, NET_NAME + '_normal_sample.png'))

In [ ]:
imgs = img.cpu().numpy()
recons_list = recons.cpu().numpy()
    
iqr, lower_bound, higher_bound = stats(list(map(pearson_correlation, img.cpu().numpy(), recons.cpu().numpy())))
print(f'IQR: {iqr}\nLower Bound: {lower_bound}\nHigher Bound: {higher_bound}')

In [ ]:
bad_mse = list(map(pearson_correlation, data.cpu().numpy(), recon.cpu().numpy()))
print(f'Mean Squared Error on Defects:\n\
    Min: {np.min(bad_mse)}\n\
    Median: {np.median(bad_mse)}\n\
    Max: {np.max(bad_mse)}')

In [ ]:
# plt.boxplot(list(map(root_mean_squared_error, img.cpu().numpy(), recons.cpu().numpy())))
# plt.show
import seaborn as sns

fig, ax = plt.subplots(1, 2)
sns.boxplot(list(map(pearson_correlation, data.cpu().numpy(), recon.cpu().numpy())), ax=ax[0])
sns.boxplot(list(map(pearson_correlation, img.cpu().numpy(), recons.cpu().numpy())), ax=ax[1])
fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
bxp = sns.boxplot(list(map(pearson_correlation, img.cpu().numpy(), recons.cpu().numpy())))
bxp.set(ylabel='Value', title=f'Pearson Correlation {NET_NAME}')
sns.swarmplot(bad_mse, dodge=True)
ax.hlines(lower_bound, -.3, .3, colors='red')
sns.despine()

In [ ]:
fig.savefig(os.path.join(GRAPH_OUT, NET_NAME + '.png'))